In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, NMF
import tensorflow as tf

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from wordcloud import WordCloud
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
train= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/train.csv')
test= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Index  120000 non-null  object
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
dtypes: object(3)
memory usage: 2.7+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class Index  7600 non-null   object
 1   Title        7600 non-null   object
 2   Description  7600 non-null   object
dtypes: object(3)
memory usage: 178.2+ KB


In [ ]:
train_copy = train.drop('Title',axis=1)
test_copy = train.drop('Title',axis=1)

In [ ]:
train_copy = train_copy.rename(columns={'Class Index':'Class'})
test_copy = test_copy.rename(columns={'Class Index':'Class'})

In [ ]:
train.isnull().values.any()

False

In [ ]:
train_copy.isnull().sum()

Class          0
Description    0
dtype: int64

In [ ]:
train_copy.groupby('Class')['Class'].count()

Class
Business    30000
Sci/Tech    30000
Sports      30000
World       30000
Name: Class, dtype: int64

In [ ]:
train_copy['Description'].head()

0    Reuters - Short-sellers, Wall Street's dwindli...
1    Reuters - Private investment firm Carlyle Grou...
2    Reuters - Soaring crude prices plus worries\ab...
3    Reuters - Authorities have halted oil export\f...
4    AFP - Tearaway world oil prices, toppling reco...
Name: Description, dtype: object

In [ ]:
# Clean the description

def Preprocess(df):

                 

  lema = WordNetLemmatizer()

  for index in df.index:
    sen = re.sub('[^a-zA-Z]',' ',df['Description'][index])
    sen = sen.lower().split()
    sen = [word for word in sen if len(word)>3]
    sen = [lema.lemmatize(word) for word in sen if word not in stopwords.words('english')]
    sen = ' '.join(sen)

    df['Description'][index]= sen

    df['Description'] = df['Description'].apply(lambda x: re.sub(' +',' ',x))
    
    # Encode the text
    le = LabelEncoder()
    df['Labels'] = le.fit_transform(df['Class'])
    

  return df

In [ ]:
train_df = Preprocess(train_copy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


KeyboardInterrupt: ignored

In [ ]:
def modeling(X, y):
  models = {
      'Logistic_reg': OneVsRestClassifier(LogisticRegression()),
      'Decisiontree': DecisionTreeClassifier(),
      'RandomForest':RandomForestClassifier(),
      'AdaBoost': AdaBoostClassifier(),
      'MultinomialNB':MultinomialNB(),
      'KNN':KNeighborsClassifier()
      
  }

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 42, shuffle=True)

  for model_name, model in models.items():
    clf = GridSearchCV(model,param_grid={}, scoring='accuracy', cv=5)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('\n')
    print(model_name)
    
    print('\n')
    print('Best Score: ')
    print(clf.best_score_)

    print('\n')
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('\n')
  
    cf_mat = confusion_matrix(y_test, y_pred)
    sns.heatmap(cf_mat, annot=True)
    plt.show()

    print('\n')
    
    print(classification_report(y_test, y_pred))
    print('End Training.....\n\n')

In [ ]:
# apply Tfidf

vec = TfidfVectorizer(ngram_range=(1,2))
X= vec.fit_transform(train_df['Description'])
print(X.shape)

X_train,X_test,y_train,y_test=train_test_split(X,train_df['Labels'],random_state=42,test_size=0.3)

clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

(20000, 257978)


0.8891666666666667

In [ ]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1464
           1       0.87      0.85      0.86      1520
           2       0.94      0.97      0.95      1487
           3       0.91      0.89      0.90      1529

    accuracy                           0.89      6000
   macro avg       0.89      0.89      0.89      6000
weighted avg       0.89      0.89      0.89      6000

